## 从零开始AI - 机器学习 - Tensorflow Keras 示例

> ··· 您可以访问 [https://github.com/HaoHoo/F02AI](https://github.com/HaoHoo/F02AI) 来获得 “从零开始AI” 系列全部的内容。··· <br>
>`请以原样使用或转发，欢迎Issue，敬请PR；如果觉得有些意思，欢迎Fork，敬谢Star。` 

在以往的示例里，我们已经通过简单易用的机器学习工作室图形界面尝试了MNIST的机器学习过程。那么在简单轻松的拖拽之下，系统到底做了些什么呢？让我们尝试Tensorflow的框架示例代码来学习一下吧。

开始之前说明一下，这里的示例代码主要来自 https://tensorflow.google.cn/tutorials/quickstart/advanced?hl=zh-cn ，酌情增加了一些代码和注释。

原本我是想找Tensorflow的示例代码的，后来发现Keras的API已经封装简化了许多代码工作，并且Keras在新的TensorFlow版本中已经集成。如果import的TensorFlow版本在2.0之前，则需要单独执行 pip import keras。如下的示例代码是基于TensorFlow 2.0的，因此有关keras的组件的import都是from自tensorflow库。如果使用之前的版本，则import的库需要从tensorflow.keras改成keras。当然，建议更新tensorflow库避免兼容问题。

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

如果没有执行Install Certificates脚本导入Python的证书，原本的示例代码运行时可能会报SSL证书验证错误。除了执行证书导入脚本之外，也可以运行以下代码，临时忽略证书验证。

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

由于 tf.keras 的 datasets 已经包含了MNIST的数据集，所以这里不需要再自行准备或从MNIST的网站上下载，只需要调用即可。如果没有对证书进行处理的话，有可能会在下载数据集文件时超时或提示证书错误。
如果下载没有问题，就可以使用 mnist.load_data() 准备好用于训练和测试的数据集。由于数据集中的像素都是基于灰度的值，所以可以除以255进行数据规范化，使之都转成位于0-1间的浮点数。


In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 看看数据加载之后，训练数据的维度
print(x_train.shape)

# 可以取消下行注释，修改队列位置0为其他，查看其中的训练数据抽样
#print(x_train[0])


# 添加一个通道维度
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# 看看添加之后，测试数据的维度。因为两部分数据处理上是一致的
print(x_test.shape)

使用 tf.data 来将数据集切分为 batch 以及混淆数据集。

深度学习使用梯度下降优化算法。为实现梯度下降，每次的参数更新有几种方式：
- 处理全部的数据集计算损失函数，计算函数对各参数的梯度并更新梯度，称之为批量梯度下降。因为要全部计算所有数据，因此速度较慢。
- 每处理一个数据就计算损失函数，然后算梯度更新参数，称之为随机梯度下降。因为结果较离散，故更新参数可能不很稳定在最佳值附近分散摇摆。
- 为折中上述两种方式，选择切分一小批数据计算，然后更新参数，这样参数相对接近稳定，也不每次需要太大的计算量。


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 查看经过混淆和切分的批量数据集
print(train_ds)
print(test_ds)

使用 Keras 模型子类化（model subclassing） API 构建 tf.keras 模型。

借助这段简洁的代码，我们可以看到构建训练模型的有四层网络：
- 首先是一个2D的卷积层，过滤器为32，卷积核为3，激活函数是 relu；
- 为了对接前后层，放置了一个扁平层，将前一层卷积层多维转为一维，然后连接后面的全连接层；
- 接着是一个全连接层，将上一层的输出转为128维，激活函数是 relu；
- 最后是一个全连接层，将上一层的输出转为10维，激活函数是 softmax。这也是最终获得0-9数字分类概率的输出层。

关于这些层定义的参数：
- Conv2D 卷积层参数可以查看：https://tensorflow.google.cn/api_docs/python/tf/keras/layers/Conv2D
- Flatten 扁平层参数可以查看：https://tensorflow.google.cn/api_docs/python/tf/keras/layers/Flatten
- Dense 全连接层参数可以查看：https://tensorflow.google.cn/api_docs/python/tf/keras/layers/Dense

如果尚不清楚这些层是如何工作的也没有关系，后续我们可以一起看看它们是如何配合一起工作的。

In [ ]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

为训练选择损失函数和优化器。

损失函数（或称目标函数、优化评分函数）用于衡量机器学习模型的好坏。交叉熵损失函数配合 sigmod 或 softmax 会有不错的效果。
优化器能够用来更新和计算影响模型训练和模型输出的网络参数，使其逼近或达到最优值，从而最小化(或最大化)损失函数E(x)。

优化器通常可以指定学习率 lr 参数，也可以调整其他参数例如 decay, momentum, nesterov 等。也可直接使用默认值。 

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

选择衡量指标来度量模型的损失值（loss）和准确率（accuracy）。这些指标在 epoch 上累积值，然后打印出整体结果。
keras 可使用 metrics 评价函数评估当前训练模型的性能。评价函数和损失函数相似，只不过评价函数的结果不会用于训练过程中。

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

使用 tf.GradientTape 来训练模型。这里定义了一个训练步骤 funtion ，需要输入的是 images 和 labels 两个参数。

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# 计算平均值训练 loss
  train_loss(loss)
# 使用 SparseCategoricalAccuracy 评估函数计算标签 labels 和预测结果 predictions 间的训练准确率
  train_accuracy(labels, predictions)

测试模型。

In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

# 计算平均值测试 loss    
  test_loss(t_loss)
# 使用 SparseCategoricalAccuracy 评估函数计算标签 labels 和预测结果 predictions 间的测试准确率
  test_accuracy(labels, predictions)

该手写数字分类器现在在此数据集 (MNIST) 上训练得到了接近 98% 的准确率（accuracy）。

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # 在下一个epoch开始时，重置评估指标
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))


你可能会收到一个 WARNING ，关于默认的浮点数类型不匹配的警告。可以忽略，也可以运行以下代码，将 keras 后端 backend 的浮点类型统一之后重新运行前一段代码。

In [ ]:
tf.keras.backend.set_floatx('float64')

关于 Keras 有个不错的中文文档：https://keras.io/zh

要了解更多信息，请阅读 TensorFlow 教程：https://tensorflow.google.cn/